In [ ]:
import os
import random
import time

import numpy as np
import torch
# from klearn_tcyclone.training_utils.args import FLAGS, ALL_FLAGS
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils import data

from klearn_tcyclone.climada.tc_tracks import TCTracks
from klearn_tcyclone.data_utils import (
    LinearScaler,
)
from klearn_tcyclone.KNF.modules.eval_metrics import RMSE_TCTracks
from klearn_tcyclone.KNF.modules.train_utils import (
    eval_epoch_koopman,
    train_epoch_koopman,
)
from klearn_tcyclone.knf_data_utils import TCTrackDataset
from absl import app, flags

from klearn_tcyclone.training_utils.training_utils import extend_by_default_flag_values

from kkseq.koopkernel_sequencer import RBFKernel, NystroemKoopKernelSequencer

from klearn_tcyclone.data_utils import (
    standardized_context_dataset_from_TCTracks,
)

from matplotlib import pyplot as plt
from klearn_tcyclone.KNF.modules.models import Koopman
import torch


from klearn_tcyclone.koopkernel_seq2seq_utils import plot_TCTrackDataset_item_2D_with_prediction_koopman_seq2seq
from klearn_tcyclone.plot_utils import plot_TCTrackDataset_item_2D

from klearn_tcyclone.koopkernel_seq2seq_utils import train_koopkernel_seq2seq_model
from klearn_tcyclone.knf_model_utils import train_KNF_model
import logging

## Import data

In [2]:
flag_params = {
    "year_range": [1980, 1988],
    "basin": "NA"
}
flag_params = extend_by_default_flag_values(flag_params)

In [4]:
# random.seed(flag_params["seed"])  # python random generator
# np.random.seed(flag_params["seed"])  # numpy random generator

# torch.manual_seed(flag_params["seed"])
# torch.cuda.manual_seed_all(flag_params["seed"])

# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# feature_list = [
#     "lon",
#     "lat",
#     "max_sustained_wind",
#     # "radius_max_wind",
#     # "radius_oci",
#     "central_pressure",
#     "environmental_pressure",
# ]

# # feature_list = [
# #     "lon",
# #     "lat",
# #     "max_sustained_wind",
# #     "radius_max_wind",
# #     "radius_oci",
# #     "central_pressure",
# #     "environmental_pressure",
# # ]

# # these are not contained as flags
# # encoder_hidden_dim = flag_params["hidden_dim"]
# # decoder_hidden_dim = flag_params["hidden_dim"]
# # encoder_num_layers = flag_params["num_layers"]
# # decoder_num_layers = flag_params["num_layers"]

# output_dim = flag_params["input_dim"]
# num_feats = len(feature_list)
# learning_rate = flag_params["learning_rate"]
# # ---------------

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Device", device)

# scaler = LinearScaler()

In [5]:
# Datasets
tc_tracks = TCTracks.from_ibtracs_netcdf(
    provider="official",
    year_range=flag_params["year_range"],
    basin=flag_params["basin"],
)


2025-02-23 20:02:12,640 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2023-06-07 23:07:38 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /home/ecjb/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.
2025-02-23 20:03:50,015 - climada.hazard.tc_tracks - WARNING - 49 storm events are discarded because no valid wind/pressure values have been found: 1980199N31284, 1980200N25270, 1980204N23287, 1980226N15339, 1980238N16328, ...


/home/ecjb/packages/climada_python/climada/hazard/tc_tracks.py:614: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ibtracs_ds.dims['storm'] == 0:


In [9]:
feature_list = [
    "lon",
    "lat",
    "max_sustained_wind",
    "radius_max_wind",
    "radius_oci",
    "central_pressure",
    "environmental_pressure",
]

### KNF

In [29]:
flag_params["batch_size"] = 32
flag_params["num_epochs"] = 20
flag_params["train_output_length"] = 6
flag_params["test_output_length"] = 6
flag_params["input_length"] = 14
flag_params["context_length"] = 15
flag_params["model"] = "KNF"
flag_params["input_dim"] = 2

# assert flag_params["context_length"] == flag_params["input_length"] + flag_params["train_output_length"]

In [74]:
if flag_params["input_length"] % flag_params["input_dim"] != 0:
    raise Exception("input_length must be divisible by input_dim")

In [75]:
random.seed(flag_params["seed"])  # python random generator
np.random.seed(flag_params["seed"])  # numpy random generator

# Logging and define save paths
# current_file_dir_path = os.path.dirname(os.path.abspath(__file__))
current_file_dir_path = os.getcwd()
results_dir = os.path.join(
    current_file_dir_path,
    "training_results",
    "{}_yrange{}_basin{}".format(
        flag_params["dataset"],
        "".join(map(str, flag_params["year_range"])),
        flag_params["basin"],
    ),
    flag_params["model"],
)
logs_dir = os.path.join(
    current_file_dir_path,
    "logs",
    # flag_params["dataset"],
    "{}_yrange{}".format(
        flag_params["dataset"],
        "".join(map(str, flag_params["year_range"])),
    ),
)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(logs_dir, flag_params["model"] + ".log"),
    encoding="utf-8",
    filemode="a",
    format="{asctime} - {name} - {filename}:{lineno} - {levelname} - {message}",
    style="{",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)
logger = logging.getLogger(flag_params["model"] + "_logger")
logger.info(flag_params)

In [76]:
tc_tracks_train, tc_tracks_test = train_test_split(tc_tracks.data, test_size=0.1, random_state=flag_params["seed"])

In [77]:
from klearn_tcyclone.KNF.modules.eval_metrics import RMSE_TCTracks

In [ ]:
model_knf = train_KNF_model(
    tc_tracks_train,
    tc_tracks_test,
    feature_list,
    flag_params=flag_params,
    logger=logger,
    results_dir=results_dir,
)

/home/ecjb/projects/koopman_learning_tropical_cyclone/klearn_tcyclone/data_utils.py:262: RuntimeWarning: divide by zero encountered in divide
  scaling_factor = (self.target_max_vec - self.target_min_vec) / (
/home/ecjb/projects/koopman_learning_tropical_cyclone/klearn_tcyclone/data_utils.py:266: RuntimeWarning: invalid value encountered in multiply
  scaled_diffs_to_min_vec = scaling_factor * diffs_to_min
/home/ecjb/projects/koopman_learning_tropical_cyclone/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
ERROR:tornado.general:SEND Error: Host unreachable


### Koopman kernel sequence model

In [15]:
flag_params["batch_size"] = 32
flag_params["num_epochs"] = 20
flag_params["train_output_length"] = 6
flag_params["test_output_length"] = 6
flag_params["input_length"] = 13
flag_params["context_length"] = 13 + 6
flag_params["model"] = "koopkernelseq"

assert flag_params["context_length"] == flag_params["input_length"] + flag_params["train_output_length"]

In [16]:
random.seed(flag_params["seed"])  # python random generator
np.random.seed(flag_params["seed"])  # numpy random generator

# Logging and define save paths
# current_file_dir_path = os.path.dirname(os.path.abspath(__file__))
current_file_dir_path = os.getcwd()
results_dir = os.path.join(
    current_file_dir_path,
    "training_results",
    "{}_yrange{}_basin{}".format(
        flag_params["dataset"],
        "".join(map(str, flag_params["year_range"])),
        flag_params["basin"],
    ),
    flag_params["model"],
)
logs_dir = os.path.join(
    current_file_dir_path,
    "logs",
    # flag_params["dataset"],
    "{}_yrange{}".format(
        flag_params["dataset"],
        "".join(map(str, flag_params["year_range"])),
    ),
)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(logs_dir, flag_params["model"] + ".log"),
    encoding="utf-8",
    filemode="a",
    format="{asctime} - {name} - {filename}:{lineno} - {levelname} - {message}",
    style="{",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)
logger = logging.getLogger(flag_params["model"] + "_logger")
logger.info(flag_params)

In [1]:
context_mode = "last_context"

In [18]:
model_koopkernelseq = train_koopkernel_seq2seq_model(
    tc_tracks,
    feature_list,
    flag_params,
    basin=flag_params["basin"],
    logger=logger,
    results_dir=results_dir,
    context_mode=context_mode,
)

intput_length and output_length are not None.
intput_length and output_length are not None.


/home/ecjb/projects/koopman_learning_tropical_cyclone/klearn_tcyclone/koopkernel_seq2seq_utils.py:60: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  tensor_context_tgts = torch.tensor(


intput_length and output_length are not None.
intput_length and output_length are not None.
eval comparison 0.4306322354171163 1000000.0
eval comparison 0.4124485288101543 0.4306322354171163
eval comparison 0.4062353576258865 0.4124485288101543
eval comparison 0.40191915941338713 0.4062353576258865
eval comparison 0.4029100931999597 0.40191915941338713
eval comparison 0.398685597798973 0.40191915941338713
eval comparison 0.38915768291113395 0.398685597798973
eval comparison 0.3894740828400979 0.38915768291113395
eval comparison 0.391562459900771 0.38915768291113395
eval comparison 0.3919878372099242 0.38915768291113395
eval comparison 0.3919211626925803 0.38915768291113395
eval comparison 0.3915622410807625 0.38915768291113395
eval comparison 0.3910512601179991 0.38915768291113395
eval comparison 0.3945513993962717 0.38915768291113395
eval comparison 0.39013076510320793 0.38915768291113395
eval comparison 0.3875762302553829 0.38915768291113395
eval comparison 0.38599908700276647 0.3875

# Testing NANs in dataset

In [18]:
"""Trains UMAP model for eigenfunction clustering."""

import os
import pickle
from random import sample
from itertools import product

import umap
from sklearn.model_selection import train_test_split

from klearn_tcyclone.climada.tc_tracks_tools import BASINS_SELECTION
from klearn_tcyclone.climada.utils import get_TCTrack_dict
from klearn_tcyclone.data_utils import (
    load_model,
    standardized_context_dataset_from_TCTracks,
)
from klearn_tcyclone.kooplearn.spectral_analysis import (
    get_df_evecs,
)
from klearn_tcyclone.training_utils.training_utils import (
    extend_by_default_flag_values,
)

from klearn_tcyclone.climada.tc_tracks import TCTracks
import numpy as np
from klearn_tcyclone.data_utils import (
    LinearScaler,
)

In [73]:
current_file_dir_path = os.getcwd()
path_training_results = os.path.join(
    current_file_dir_path,
    "../../train_models/",
    "training_results",
)

model_str = "Nystroem_RRR"


time_lag = 1
slide_by = 1
basins = BASINS_SELECTION
feature_list = [
    "lon",
    "lat",
    "max_sustained_wind",
    "radius_max_wind",
    "radius_oci",
    "central_pressure",
    "environmental_pressure",
]


flag_params = {
    # "year_range": [1980, 1988],
    # "year_range": [1980, 2021],
    "year_range": [1990, 2000],
    # "year_range": [2010, 2021],
    "basin": "NA"
}
flag_params = extend_by_default_flag_values(flag_params)

# flag_params = {
#     "koopman_kernel_num_train_stops": 10,
#     "year_range": [1980, 2021],
#     "model": model_str,
#     "koopman_kernel_length_scale": 10.0,
#     "koopman_kernel_rank": 50,
#     "koopman_kernel_num_centers": 800,
#     "context_length": 16,
#     "koopman_kernel_reduced_rank": False,
#     "time_step_h": 1.0,
# }
flag_params = extend_by_default_flag_values(flag_params)

model_config = {
    "length_scale": 10.0,
    "reduced_rank": False,
    "rank": 50,
    "num_centers": 600,
    "tikhonov_reg": 1e-8,
    "svd_solver": "arnoldi",
    "rng_seed": 42,
}

In [74]:
scaler = LinearScaler()

In [77]:
basins = [flag_params["basin"]]

In [76]:
# Datasets
tc_tracks = TCTracks.from_ibtracs_netcdf(
    provider="official",
    year_range=flag_params["year_range"],
    basin=flag_params["basin"],
)
time_step_h = 1.0
tc_tracks.equal_timestep(time_step_h=time_step_h)

tc_tracks_train, tc_tracks_test = train_test_split(tc_tracks.data, test_size=0.1, random_state=flag_params["seed"])

2025-02-23 20:27:03,930 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2023-06-07 23:07:38 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /home/ecjb/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.
2025-02-23 20:28:59,087 - climada.hazard.tc_tracks - WARNING - 4 storm events are discarded because no valid wind/pressure values have been found: 1990145N19276, 1990262N15315, 1991187N23264, 1991241N11329.


/home/ecjb/packages/climada_python/climada/hazard/tc_tracks.py:614: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ibtracs_ds.dims['storm'] == 0:


In [ ]:
# Datasets
tc_tracks_dict = get_TCTrack_dict(
    basins=basins,
    time_step_h=flag_params["time_step_h"],
    year_range=flag_params["year_range"],
)
for basin in basins:
    tc_tracks_train, tc_tracks_test = train_test_split(
        tc_tracks_dict[basin].data, test_size=0.1, random_state=flag_params["seed"]
    )
    # tc_tracks_dict[basin] = {
    #     "train": tc_tracks_train,
    #     "test": tc_tracks_test,
    # }

2025-02-23 20:20:02,857 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2023-06-07 23:07:38 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /home/ecjb/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.


In [78]:
idx = np.random.randint(0, len(tc_tracks_train))
tc_tracks_train[idx].to_dataframe()

,radius_max_wind,radius_oci,max_sustained_wind,central_pressure,environmental_pressure,time_step,basin,lon,lat
time,,,,,,,,,
1999-10-04 06:00:00,0.0,0.0,30.0,1005.000000,1010.0,1.0,NA,-94.099998,20.100000
1999-10-04 07:00:00,0.0,0.0,30.0,1004.833333,1010.0,1.0,NA,-94.087736,20.006465
1999-10-04 08:00:00,0.0,0.0,30.0,1004.666667,1010.0,1.0,NA,-94.070191,19.905172
1999-10-04 09:00:00,0.0,0.0,30.0,1004.500000,1010.0,1.0,NA,-94.050003,19.799999
1999-10-04 10:00:00,0.0,0.0,30.0,1004.333333,1010.0,1.0,NA,-94.029815,19.694827
1999-10-04 11:00:00,0.0,0.0,30.0,1004.166667,1010.0,1.0,NA,-94.012266,19.593534
1999-10-04 12:00:00,0.0,0.0,30.0,1004.000000,1010.0,1.0,NA,-94.000000,19.500000
1999-10-04 13:00:00,0.0,0.0,30.0,1003.833333,1010.0,1.0,NA,-93.994867,19.416076
1999-10-04 14:00:00,0.0,0.0,30.0,1003.666667,1010.0,1.0,NA,-93.995565,19.335504


In [79]:
tensor_context_train_standardized = standardized_context_dataset_from_TCTracks(
    tc_tracks_train,
    feature_list=feature_list,
    scaler=scaler,
    context_length=flag_params["context_length"],
    time_lag=1,
    fit=True,
    periodic_shift=True,
    basin=flag_params["basin"],
    input_length=flag_params["input_length"],
    output_length=flag_params["train_output_length"],
)

intput_length and output_length are not None.


/home/ecjb/projects/koopman_learning_tropical_cyclone/klearn_tcyclone/data_utils.py:262: RuntimeWarning: divide by zero encountered in divide
  scaling_factor = (self.target_max_vec - self.target_min_vec) / (
/home/ecjb/projects/koopman_learning_tropical_cyclone/klearn_tcyclone/data_utils.py:266: RuntimeWarning: invalid value encountered in multiply
  scaled_diffs_to_min_vec = scaling_factor * diffs_to_min


In [80]:
tensor_context_train_standardized.data.shape

(18074, 31, 7)

In [82]:
idx = np.random.randint(0, tensor_context_train_standardized.shape[0])
tensor_context_train_standardized.data[idx, :, 3]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan])